In [ ]:
import pandas as pd
import numpy as np
from cobra import Model, Reaction, Metabolite

model = Model()

# Define reaction order
order = ['GlycP', 'GPM', 'PGI', 'ATPPFK', 'FBPald', 'TPI', 'GAPDH', 'PGK', 
         'TktA', 'SBPald', 'SBPase', 'TktB', 'RibE', 'RibI', 'RbuK', 'RbuCO', 
         'PGM', 'ENO', 'PYK', 'PDH', 'ACS', 'Thio', 'AAR']

rxns_table = pd.read_csv('fluxes_labeling_for_MDF.csv', delimiter=',')
# Keep only the rxns in the ORDER list
rxns_table = rxns_table[rxns_table['Reaction Name'].isin(order)]

# Sort by the ORDER list
rxns_table['sort_key'] = rxns_table['Reaction Name'].apply(lambda x: order.index(x))
rxns_table = rxns_table.sort_values('sort_key').drop('sort_key', axis=1)

rt = rxns_table.set_index('Reaction Name', drop=False)
rt = rt.reindex(order, fill_value=0)
rt['Reaction Formula'] = rt['Reaction Formula'].astype(str).replace('0','')
rt['Relative Flux']     = rt['Relative Flux'].astype(float).fillna(0)
rxns_table = rt.reset_index(drop=True)

for _, row in rxns_table.iterrows():
    
    rxn = Reaction(row['Reaction Name'])
    
    model.add_reactions([rxn])
    
    rxn.reaction = row['Reaction Formula']

# Read standard dG values
dGs_table = pd.read_csv('standard_dGs.csv', delimiter='\t')
dGs_table = dGs_table[dGs_table['reactions'].isin(order)]
dGs_table['sort_key'] = dGs_table['reactions'].apply(lambda x: order.index(x))
dGs_table = dGs_table.sort_values('sort_key').drop('sort_key', axis=1)
list_dGs = dGs_table.iloc[:, 1].values
R = 0.008314; T = 293.15
list_Keqs = np.exp(-list_dGs / (R * T))

# Read concentrations
conc_table = pd.read_csv('compound_concentrations.csv', delimiter='\t')
conc_table = conc_table[conc_table['Compound'].isin([m.id for m in model.metabolites])]
conc_table['sort_key'] = conc_table['Compound'].apply(lambda x: model.metabolites.index(x))
conc_table = conc_table.sort_values('sort_key').drop('sort_key', axis=1)
concs_MDF = conc_table['Concentration (M)'].values  

# Initialize matrices
num_mets = len(model.metabolites)
num_rxns = len(model.reactions)
KMS = np.zeros((num_mets, num_rxns))
stoich_sub = np.zeros((num_mets, num_rxns))
KMP = np.zeros((num_mets, num_rxns))
stoich_prod = np.zeros((num_mets, num_rxns))

# Fill matrices
for rxn_idx, rxn in enumerate(model.reactions):
    for met in rxn.metabolites:
        coeff   = rxn.metabolites[met]
        met_idx = list(model.metabolites).index(met)
        # Use met.id, not met.name, to detect H2O
        is_water = (met.id == 'H2O') or met.id.startswith('H2O_')

        if coeff < 0:
            # substrate
            if is_water:
                KMS_val = 1.0
            else:
                KMS_val = concs_MDF[met_idx] * 1000
            KMS[met_idx, rxn_idx]     = KMS_val
            stoich_sub[met_idx, rxn_idx] = abs(coeff)
        else:
            # product
            if is_water:
                KMP_val = 1.0
            else:
                KMP_val = concs_MDF[met_idx] * 1000
            KMP[met_idx, rxn_idx]      = KMP_val
            stoich_prod[met_idx, rxn_idx] = abs(coeff)

adjusted_substrates = []
adjusted_products = []
for rxn_idx in range(num_rxns):
    subs = KMS[:, rxn_idx][KMS[:, rxn_idx] != 0] ** stoich_sub[:, rxn_idx][KMS[:, rxn_idx] != 0]
    prods = KMP[:, rxn_idx][KMP[:, rxn_idx] != 0] ** stoich_prod[:, rxn_idx][KMP[:, rxn_idx] != 0]
    adjusted_substrates.append(np.prod(subs))
    adjusted_products.append(np.prod(prods))

protein_table = pd.read_csv('input_MW_values.csv')
kcatF = np.zeros(num_rxns)
kcatR = np.zeros(num_rxns)
weights = np.zeros(num_rxns)

for rxn_idx, rxn in enumerate(model.reactions):
    row = protein_table[protein_table['protein'] == rxn.id].iloc[0]
    weights[rxn_idx] = row['MW']
    kcatF[rxn_idx] = 36.72
    kcatR[rxn_idx] = (kcatF[rxn_idx] * adjusted_products[rxn_idx]) / (list_Keqs[rxn_idx] * adjusted_substrates[rxn_idx])
    

KMs_list = []
for rxn_idx in range(num_rxns):
    mets = []
    for met_idx in range(num_mets):
        val = KMS[met_idx, rxn_idx] + KMP[met_idx, rxn_idx]
        if val != 0:
            mets.append(f"{model.metabolites[met_idx].id}:{val}")
    KMs_list.append(' '.join(mets))

T1 = pd.DataFrame({
    'Reaction Name': [rxn.id for rxn in model.reactions],
    'kcatf (1/s)': kcatF,
    'kcatr (1/s)': kcatR,
    'KM (mM)': KMs_list,
    'MWe(Da)': weights
})

T1['sort_key'] = T1['Reaction Name'].apply(lambda x: order.index(x))
T1 = T1.sort_values('sort_key').drop('sort_key', axis=1)
T1S = T1

for col in ['kcatf (1/s)', 'kcatr (1/s)', 'MWe(Da)']:
    T1[col] = T1[col].apply(lambda x: f"{x:.2e}")

def format_kms(km_str):
    parts = km_str.split()
    formatted_parts = []
    for part in parts:
        try:
            met, val = part.split(':')
            val = float(val)
            formatted_parts.append(f"{met}:{val:.2e}")
        except:
            formatted_parts.append(part)  # leave unchanged if malformed
    return ' '.join(formatted_parts)

T1['KM (mM)'] = T1['KM (mM)'].apply(format_kms)

T1.to_csv('consistent_kinetic_parameters.csv', index=False)

T2 = pd.DataFrame({'Reaction Formula': rxns_table['Reaction Formula'].values})

T3 = pd.DataFrame({'Relative Flux': rxns_table['Relative Flux'].values})

# Normalize fluxes using ACS reaction
acs_mask = T1S['Reaction Name'] == 'ACS'
if not any(acs_mask):
    raise ValueError("Normalizer reaction 'ACS' not found in T1")

flux_ACS = T3.loc[acs_mask, 'Relative Flux'].values[0]
T3['Relative Flux'] = rxns_table['Relative Flux'] / flux_ACS

T5 = pd.concat([
    T2.reset_index(drop=True),
    T3.reset_index(drop=True),
    T1S.reset_index(drop=True)
], axis=1)

T5 = T5.rename(columns={
    'kcatf (1/s)': 'kcrf(1/s)',
    'kcatr (1/s)': 'kcrr(1/s)',
    'KM (mM)': 'kM(mM)'
})

# Sort by original order
T5['sort_key'] = T5['Reaction Name'].apply(lambda x: order.index(x))
T5 = T5.sort_values('sort_key').drop('sort_key', axis=1)

# Save final output
desired_cols = [
    'Reaction Name',
    'Reaction Formula',
    'Relative Flux',
    'kcrf(1/s)',
    'kcrr(1/s)',
    'kM(mM)',
    'MWe(Da)'
]
# Reindex columns
T5 = T5.reindex(columns=desired_cols)
T5.to_csv('file_for_ECM.csv', index=False)